In [108]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)

print("Librerías cargadas correctamente")


Librerías cargadas correctamente


In [109]:
COVID_DATA_PATH = "data/covid_19_data.csv"
print(f"Leyendo datos principales desde {COVID_DATA_PATH}")

df_raw = pd.read_csv(COVID_DATA_PATH)
print(f"Registros cargados: {len(df_raw):,}")
df_raw.head()


Leyendo datos principales desde data/covid_19_data.csv
Registros cargados: 306,429


,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [110]:
cols_to_sum = [col for col in ["Confirmed", "Deaths", "Recovered"] if col in df_raw.columns]

if cols_to_sum:
    df = (
        df_raw.groupby(["Country/Region", "ObservationDate"], as_index=False)[cols_to_sum]
              .sum()
    )
else:
    df = df_raw.groupby(["Country/Region", "ObservationDate"], as_index=False).first()

print(f"Registros agregados por país y fecha: {len(df):,}")
df.head()


Registros agregados por país y fecha: 87,281


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered
0,Azerbaijan,02/28/2020,1.0,0.0,0.0
1,"('St. Martin',)",03/10/2020,2.0,0.0,0.0
2,Afghanistan,01/01/2021,51526.0,2191.0,41727.0
3,Afghanistan,01/02/2021,51526.0,2191.0,41727.0
4,Afghanistan,01/03/2021,51526.0,2191.0,41727.0


In [111]:
COORD_DATA_PATH = "data/time_series_covid_19_confirmed.csv"
print(f"Leyendo coordenadas base desde {COORD_DATA_PATH}")

df_coords_raw = pd.read_csv(COORD_DATA_PATH)
print(f"Registros con coordenadas: {len(df_coords_raw):,}")
df_coords_raw[['Country/Region', 'Lat', 'Long']].head()


Leyendo coordenadas base desde data/time_series_covid_19_confirmed.csv
Registros con coordenadas: 276


,Country/Region,Lat,Long
0,Afghanistan,33.93911,67.709953
1,Albania,41.15330,20.168300
2,Algeria,28.03390,1.659600
3,Andorra,42.50630,1.521800
4,Angola,-11.20270,17.873900


In [112]:
coords_by_country = (
    df_coords_raw.groupby('Country/Region')[['Lat', 'Long']]
                 .mean()
                 .reset_index()
                 .rename(columns={'Lat': 'Lat_prom', 'Long': 'Long_prom'})
)

print(f"Países con coordenadas disponibles: {len(coords_by_country):,}")
coords_by_country.head()


Países con coordenadas disponibles: 193


,Country/Region,Lat_prom,Long_prom
0,Afghanistan,33.93911,67.709953
1,Albania,41.15330,20.168300
2,Algeria,28.03390,1.659600
3,Andorra,42.50630,1.521800
4,Angola,-11.20270,17.873900


In [113]:
def clean_country_name(name: str) -> str:
    if pd.isna(name):
        return name
    # Maneja strings tipo "('St. Martin',)"
    if isinstance(name, str) and name.startswith("('") and name.endswith("',)"):
        name = name[2:-3]
    return name.strip()

coords_by_country['Country/Region'] = coords_by_country['Country/Region'].apply(clean_country_name)
df['Country/Region'] = df['Country/Region'].apply(clean_country_name)

df_missing_coords = df[~df['Country/Region'].isin(coords_by_country['Country/Region'])]['Country/Region'].unique()
print(f"Países sin coordenadas tras la limpieza: {len(df_missing_coords)}")
print(df_missing_coords[:10])


Países sin coordenadas tras la limpieza: 39
['St. Martin' 'Aruba' 'Bahamas, The' 'Cape Verde' 'Cayman Islands'
 'Channel Islands' 'Curacao' 'Czech Republic' 'East Timor' 'Faroe Islands']


In [114]:
country_aliases = {
    "US": "US",
    "United States": "US",
    "Mainland China": "China",
    "Taiwan": "Taiwan*",
    "Taiwan*": "Taiwan*",
    "South Korea": "Korea, South",
    "Korea, South": "Korea, South",
    "Congo (Kinshasa)": "Congo (Kinshasa)",
    "Congo (Brazzaville)": "Congo (Brazzaville)",
    "Republic of the Congo": "Congo (Brazzaville)",
    "Cape Verde": "Cabo Verde",
    "Cabo Verde": "Cabo Verde",
    "Burma": "Burma",
    "Czech Republic": "Czechia",
    "East Timor": "Timor-Leste",
    "Timor-Leste": "Timor-Leste",
    "Laos": "Laos",
    "Holy See": "Holy See",
    "Vatican City": "Holy See",
    "West Bank and Gaza": "West Bank and Gaza",
    "Palestine": "West Bank and Gaza",
    "occupied Palestinian territory": "West Bank and Gaza",
    "Gambia, The": "Gambia",
    "The Gambia": "Gambia",
    "Bahamas, The": "Bahamas",
    "The Bahamas": "Bahamas",
    "Cabo Verde": "Cape Verde",
    "Guinea-Bissau": "Guinea-Bissau",
    "North Macedonia": "North Macedonia",
    "Republic of Ireland": "Ireland",
    "Ireland": "Ireland",
    "UK": "United Kingdom",
    "North Ireland": "United Kingdom",
    "St. Martin": "Saint Martin",
    "Diamond Princess": "Diamond Princess",
    "Grand Princess": "Grand Princess",
    "MS Zaandam": "MS Zaandam",
    "Cruise Ship": "Cruise Ship",
    "Summer Olympics 2020": "Summer Olympics 2020",
    "Winter Olympics 2022": "Winter Olympics 2022"
}

alias_rows = []
for alias_name, target_name in country_aliases.items():
    target_row = coords_by_country.loc[coords_by_country['Country/Region'] == target_name]
    if not target_row.empty:
        new_row = target_row.iloc[0].copy()
        new_row['Country/Region'] = alias_name
        alias_rows.append(new_row)

coords_alias = pd.DataFrame(alias_rows)

coords_combined = pd.concat([
    coords_by_country,
    coords_alias
], ignore_index=True).drop_duplicates(subset=['Country/Region'], keep='first')

missing_after_alias = df[~df['Country/Region'].isin(coords_combined['Country/Region'])]['Country/Region'].unique()
print(f"Países sin coordenadas después de aplicar alias: {len(missing_after_alias)}")
print(missing_after_alias[:10])


Países sin coordenadas después de aplicar alias: 22
['St. Martin' 'Aruba' 'Cayman Islands' 'Channel Islands' 'Curacao'
 'Faroe Islands' 'French Guiana' 'Gibraltar' 'Greenland' 'Guadeloupe']


In [115]:
manual_coords = {
    "Aruba": (12.5211, -69.9683),
    "Bonaire": (12.1784, -68.2385),
    "British Virgin Islands": (18.4207, -64.6399),
    "Cape Verde": (15.1111, -23.6167),
    "Cayman Islands": (19.3133, -81.2546),
    "Channel Islands": (49.3723, -2.3644),
    "Curacao": (12.1696, -68.9900),
    "Diamond Princess": (35.4437, 139.6380),
    "Faroe Islands": (61.8926, -6.9118),
    "French Guiana": (3.9339, -53.1258),
    "Gibraltar": (36.1408, -5.3536),
    "Greenland": (71.7069, -42.6043),
    "Guadeloupe": (16.2650, -61.5500),
    "Guam": (13.4443, 144.7937),
    "Guernsey": (49.4482, -2.5895),
    "Hong Kong": (22.3193, 114.1694),
    "Ivory Coast": (7.5399, -5.5471),
    "Jersey": (49.2138, -2.1358),
    "Kosovo": (42.6026, 20.9030),
    "Macau": (22.1987, 113.5439),
    "Mainland China": (35.8617, 104.1954),
    "Martinique": (14.6415, -61.0242),
    "Mayotte": (-12.8275, 45.1662),
    "MS Zaandam": (-33.4489, -70.6693),
    "Micronesia": (7.4256, 150.5508),
    "Montserrat": (16.7425, -62.1874),
    "Others": (0.0, 0.0),
    "Palestine": (31.9522, 35.2332),
    "Puerto Rico": (18.2208, -66.5901),
    "Republic of the Congo": (-0.2280, 15.8277),
    "Reunion": (-21.1151, 55.5364),
    "Saint Barthelemy": (17.9000, -62.8333),
    "Saint Martin": (18.0708, -63.0501),
    "Saint Pierre and Miquelon": (46.8852, -56.3159),
    "Cruise Ship": (18.3358, -64.8963),
    "Summer Olympics 2020": (35.6762, 139.6503),
    "The Bahamas": (25.0343, -77.3963),
    "The Gambia": (13.4432, -15.3101),
    "Timor-Leste": (-8.8742, 125.7275),
    "UK": (55.3781, -3.4360),
    "Vatican City": (41.9029, 12.4534),
    "Winter Olympics 2022": (39.9042, 116.4074),
    "Western Sahara": (24.2155, -12.8858)
}

manual_df = (pd.DataFrame.from_dict(manual_coords, orient='index', columns=['Lat_prom', 'Long_prom'])
             .reset_index()
             .rename(columns={'index': 'Country/Region'}))

coords_full = pd.concat([coords_combined, manual_df], ignore_index=True)
coords_full = coords_full.drop_duplicates(subset=['Country/Region'], keep='first')

still_missing = df[~df['Country/Region'].isin(coords_full['Country/Region'])]['Country/Region'].unique()
print(f"Países sin coordenadas después de los manuales: {len(still_missing)}")
if len(still_missing) > 0:
    print(still_missing)
else:
    print("Todos los países tienen coordenadas asignadas.")


Países sin coordenadas después de los manuales: 1
['St. Martin']


In [116]:
df_with_coords = df.merge(coords_full, on='Country/Region', how='left')

missing_after_merge = df_with_coords[df_with_coords['Lat_prom'].isna()]['Country/Region'].unique()
print(f"Filas sin coordenadas después del merge: {len(missing_after_merge)}")
if len(missing_after_merge) > 0:
    print(missing_after_merge)

df_with_coords.head()


Filas sin coordenadas después del merge: 1
['St. Martin']


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered,Lat_prom,Long_prom
0,Azerbaijan,02/28/2020,1.0,0.0,0.0,40.14310,47.576900
1,St. Martin,03/10/2020,2.0,0.0,0.0,NaN,NaN
2,Afghanistan,01/01/2021,51526.0,2191.0,41727.0,33.93911,67.709953
3,Afghanistan,01/02/2021,51526.0,2191.0,41727.0,33.93911,67.709953
4,Afghanistan,01/03/2021,51526.0,2191.0,41727.0,33.93911,67.709953


In [117]:
df_with_coords = df_with_coords.rename(columns={'Lat_prom': 'Latitud_promedio', 'Long_prom': 'Longitud_promedio'})

df_with_coords.head()


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered,Latitud_promedio,Longitud_promedio
0,Azerbaijan,02/28/2020,1.0,0.0,0.0,40.14310,47.576900
1,St. Martin,03/10/2020,2.0,0.0,0.0,NaN,NaN
2,Afghanistan,01/01/2021,51526.0,2191.0,41727.0,33.93911,67.709953
3,Afghanistan,01/02/2021,51526.0,2191.0,41727.0,33.93911,67.709953
4,Afghanistan,01/03/2021,51526.0,2191.0,41727.0,33.93911,67.709953


In [118]:
df_with_coords.sample(5, random_state=7)[['Country/Region', 'ObservationDate', 'Latitud_promedio', 'Longitud_promedio']]


,Country/Region,ObservationDate,Latitud_promedio,Longitud_promedio
79903,Trinidad and Tobago,03/06/2021,10.6918,-61.2225
5972,Bangladesh,02/19/2021,23.6850,90.3563
44034,Lebanon,03/18/2020,33.8547,35.8623
10632,Brazil,05/27/2021,-14.2350,-51.9253
35848,Indonesia,04/16/2021,-0.7893,113.9213


In [119]:
df_with_coords = df_with_coords.dropna(subset=['Latitud_promedio', 'Longitud_promedio'])


In [120]:
OUTPUT_PATH = "data/covid_19_country_daily_with_coords.csv"
df_with_coords.to_csv(OUTPUT_PATH, index=False)
print(f"Archivo final guardado en {OUTPUT_PATH}")


Archivo final guardado en data/covid_19_country_daily_with_coords.csv


In [121]:
summary = {
    'filas_totales': len(df_with_coords),
    'pais_unicos': df_with_coords['Country/Region'].nunique(),
    'fechas_unicas': df_with_coords['ObservationDate'].nunique(),
    'coordenadas_faltantes': int(df_with_coords['Latitud_promedio'].isna().sum())
}
summary


{'filas_totales': 87279,
 'pais_unicos': 226,
 'fechas_unicas': 494,
 'coordenadas_faltantes': 0}